In [42]:
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode
import spacy
import nltk
import stanza
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.graph_objects as go
from transformers import AutoTokenizer
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud
import seaborn as sns
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
from tabulate import tabulate  # opcional si quieres una tabla más bonita

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     -------------------- ------------------- 6.6/12.9 MB 36.2 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 38.4 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [45]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [46]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
stop_words = set(stopwords.words('spanish'))

In [48]:
file_path = "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/"


In [49]:
file_names = [
    "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/Discuros Gabriel García Marquez.txt",
    "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/Discurso Albert Camus.txt",
    "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/Discurso Bob Dylan.txt",
    "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP\Clase_5/Discurso Vargas Llosa.txt"
]

In [50]:
data = []

for name in file_names:
    ruta_completa = os.path.join(file_path, name)
    
    try:
        # Primero intento con utf-8
        try:
            with open(ruta_completa, "r", encoding="utf-8") as f:
                contenido = f.read()
        except UnicodeDecodeError:
            # Si falla, reintento con latin1
            with open(ruta_completa, "r", encoding="latin1") as f:
                contenido = f.read()
        
        data.append((name, contenido))
    
    except FileNotFoundError:
        print(f"❌ No se encontró el archivo {ruta_completa}")
    except Exception as e:
        print(f"⚠️ Error con {ruta_completa}: {e}")

df = pd.DataFrame(data, columns=["nombre_archivo", "contenido"])
print(tabulate(df, headers='keys', tablefmt='fancy_grid', showindex=False))
stop_words = set(stopwords.words('spanish'))


╒═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [53]:
def preprocess_text(text):
    # Normalizar caracteres Unicode (ej. á → a)
    text = unicodedata.normalize("NFKD", text)
    text = "".join([c for c in text if not unicodedata.combining(c)])

    # Procesar con spaCy
    doc = nlp(text.lower())

    # Extraer tokens, eliminar stopwords y puntuación
    tokens = [
        token.text for token in doc
        if token.text.strip() != ""               # elimina espacios y saltos de línea
        and token.text != "\n"                   # elimina tokens de salto explícito
        and token.text not in stop_words         # elimina stopwords
        and not token.is_punct                   # elimina puntuación
    ]

    return tokens

In [54]:
df["clear_text"] = df["contenido"].apply(preprocess_text)

In [55]:
df.head()

,nombre_archivo,contenido,clear_text
0,C:/Users/ACER/OneDrive/Documentos/Analitica de...,Discurso de aceptación del Premio Nobel\nLa so...,"[discurso, aceptacion, premio, nobel, soledad,..."
1,C:/Users/ACER/OneDrive/Documentos/Analitica de...,«Al recibir la distinción con que ha querido h...,"[recibir, distincion, querido, honrarme, libre..."
2,C:/Users/ACER/OneDrive/Documentos/Analitica de...,"Discurso en el banquete del Nobel, 10 de dicie...","[discurso, banquete, nobel, 10, diciembre, 201..."
3,C:/Users/ACER/OneDrive/Documentos/Analitica de...,Elogio de la lectura y la ficción\nAprendí a l...,"[elogio, lectura, ficcion, aprendi, leer, cinc..."


In [56]:
# 1. Convertir las listas de tokens en texto (string)
df['clear_text_str'] = df['clear_text'].apply(lambda x: " ".join(x))

# 2. Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# 3. Ajustar y transformar
tfidf_matrix = vectorizer.fit_transform(df['clear_text_str'])

# 4. Convertir a DataFrame para visualizar
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    index=df['nombre_archivo'],
    columns=vectorizer.get_feature_names_out()
)

# Mostrar las primeras filas
tfidf_df.head()

,000,10,100,12,120,16,17,1970,1979,1982,...,voz,vuelo,vuelve,vuelven,vulnerable,washington,william,xiaobo,xvi,york
nombre_archivo,,,,,,,,,,,,,,,,,,,,,
C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/Discuros Gabriel García Marquez.txt,0.000000,0.019237,0.024399,0.024399,0.024399,0.024399,0.024399,0.024399,0.024399,0.024399,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019237,0.000000,0.024399,0.019237
C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/Discurso Albert Camus.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.030556,0.000000,0.000000,0.000000,0.000000,0.000000
C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_5/Discurso Bob Dylan.txt,0.046781,0.036883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036883,0.000000,0.000000,0.000000
C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP\Clase_5/Discurso Vargas Llosa.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012428,0.012428,0.024856,0.024856,0.000000,0.012428,0.000000,0.012428,0.000000,0.009798
